In [114]:
# NOTEBOOK 03: This notebook imports the two cleaned datasets for smoking and education and merges them

In [115]:
# Importing necessary libraries for the analysis
import pandas as pd

In [116]:
## PART 1: importing data (Nils)

# importing the dataset
educ_clean = pd.read_csv(r"C:\Users\nilsr\Desktop\Uni\Masters\z.Python\Python-Introduction-Smoking-Prevalence\data\clean\clean_educ_df.csv", index_col=0)
smoking_clean = pd.read_csv(r"C:\Users\nilsr\Desktop\Uni\Masters\z.Python\Python-Introduction-Smoking-Prevalence\data\clean\clean_smoking_df.csv", index_col=0)


In [117]:
## PART 2: Prepare two datasets for merging

# Rename different variables in both datasets coherently
renames_educ = {
    "geo": "country",
    "time_period": "year",
    "obs_value": "educ_esl_pct" # ESL = Early School Leavers
}

renames_smoke = {
    "geo": "country",
    "time_period": "year",
    "obs_value": "smoke_pct"
}

educ_clean = educ_clean.rename(columns=renames_educ)
smoking_clean = smoking_clean.rename(columns=renames_smoke)

# Drop unnecessary variables
educ_clean = educ_clean.drop(columns=["age", "structure_name", "unit_of_measure"])
smoking_clean = smoking_clean.drop(columns=["age", "structure_name", "unit_of_measure"])

# Make sure key variables are equally coded / no whitespaces
for df in [educ_clean, smoking_clean]:
    df["country"] = df["country"].str.strip()
    df["sex"] = df["sex"].str.strip()


In [118]:
## PART 3: Merge the two datasets

# Check if year + country + sex are id (no duplicates exist) in both datasets
assert educ_clean[["country", "year", "sex"]].duplicated().sum() == 0
assert smoking_clean[["country", "year", "sex"]].duplicated().sum() == 0

# Merge using year + country + sex as key
educ_smoke_merged = educ_clean.merge(
    smoking_clean,
    on=["country", "year", "sex"],
    how="outer",
    validate="one_to_one",
    indicator=True
)


In [119]:
## PART 4: Inspect non-merged observations and handle missing values

educ_smoke_merged[educ_smoke_merged["_merge"] != "both"]
## This indicates smoke data is missing for 7 non-EU countries. 
## We drop these observations, as we will not be able to run multivariate analysis for them, and there is no sensible way to interpolate the data

# Keep only rows that are in both datasets
educ_smoke_merged = educ_smoke_merged[educ_smoke_merged["_merge"] == "both"]

# Optionally drop the _merge variable
educ_smoke_merged = educ_smoke_merged.drop(columns="_merge")

In [120]:
# Check for missing values
educ_smoke_merged.isna().sum()
## One missing value for educ, but was matched

# Display
educ_smoke_merged[educ_smoke_merged.isna().any(axis=1)]
## The missing value is Luxembourg, Female which is not recorded in the data. We can therefore only run the analysis for Luxembourgish men.

# Drop this observation
educ_smoke_merged = educ_smoke_merged.dropna(subset=["educ_esl_pct"])

# Assert there are no missing values left
assert educ_smoke_merged.isna().sum().sum() == 0

In [ ]:
## PART 4: Export merged dataset (Nils)

# Nicely order merged dataframe
educ_smoke_merged = educ_smoke_merged[
    ["country", "sex", "year", "educ_esl_pct", "smoke_pct"]
]

# Sort by country, then sex
educ_smoke_merged = educ_smoke_merged.sort_values(by=["country", "year"])

# Display to check
educ_smoke_merged.head()

# Export (Nils)
educ_smoke_merged.to_csv(r"C:\Users\nilsr\Desktop\Uni\Masters\z.Python\Python-Introduction-Smoking-Prevalence\data\clean\educ_smoke_merged.csv")